In [105]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import pandas as pd
import os
import time
import itertools
import lightkurve
import pickle
import sys
sys.path.append('/home/parsellsx/SPOcc/spocc/')
from loaders import * # Daniel's loaders.py file - import all functions for loading in LCs
from cesium import featurize

In [2]:
# Mount the GCP filesystem onto this VM
data_dir = "/home/parsellsx/tesslcs/"
os.system(f"gcsfuse --implicit-dirs tess-goddard-lcs {data_dir}")

0

## Sector 14 Kepler EBs Lightcurve Loading and Featurization
In this notebook, I am going to load in (from the file "cleaned_sector_14_ebs_filepaths.csv", which was itself made by "Kepler EBs to TIC IDs and Filepaths.ipynb") all the filepaths and camera data for my 1809 sector 14 eclipsing binaries. I will also identify some non-EB stars to include in the training data. I will then apply some features that I will have chosen using Cesium (cesium-ml.org) to all the lightcurves (EBs and non-EBs) and prepare to fit a model. The end state of this notebook should be that I have a file with the features of every one of the 1809 sector 14 EB light curves as well as all the non-EB sector 14 light curves that I select, and I can use this file as training data to fit a model in a separate notebook.

In [2]:
eb_path = '~/berkeley-seti/cleaned_sector_14_ebs_filepaths.csv'
ebs = pd.read_csv(eb_path,index_col=False)

In [3]:
ebs

,filename,RA,dec,TIC ID,sector,camera,CCD,mag
0,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,284.417796,48.680907,267667898,14,2,3,13.4082
1,tesslcs_sector_14_104/2_min_cadence_targets/te...,298.828625,44.489136,268305489,14,2,4,10.8095
2,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,296.839695,43.408060,272716209,14,2,4,13.2177
3,tesslcs_sector_14_104/tesslcs_tmag_12_13/tessl...,300.975737,44.112761,185057845,14,2,4,12.4576
4,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,295.940920,44.578345,271970252,14,2,4,13.5418
...,...,...,...,...,...,...,...,...
1804,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,279.856103,43.667885,383752517,14,2,3,14.2124
1805,tesslcs_sector_14_104/tesslcs_tmag_12_13/tessl...,287.856867,44.102747,158629066,14,2,3,12.2402
1806,tesslcs_sector_14_104/tesslcs_tmag_12_13/tessl...,295.573993,39.700610,184089092,14,1,1,12.5710
1807,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,288.869256,47.369978,158840322,14,2,3,13.7675


In [4]:
# Add "/home/parsellsx/tesslcs" to the beginning of every one of the filenames in the dataframe
# Note: You CANNOT do "~/tesslcs" - if you do, it doesn't recognize the files
lccoll = LightCurveCollection(list(ebs['sector']),list(ebs['camera']),['/home/parsellsx/tesslcs/' + s for s in list(ebs['filename'])],factor=10)

In [5]:
lccoll

After much pain, I finally got the LightCurveCollection loaded! It was giving me grief with the "Note:" that I added above, but it's in here now. But that LC collection has only the EBs in it - I also need to load in some non-EBs. These are going to be identified with Brian Powell's catalog, because we're assuming that pretty much EB that TESS observed is listed in there. So all I need to do is get the sector 14 LCs that _aren't_ in his catalog, then take a random sample of, say, 1500-2000 of those, and combine that dataframe with the one above and load in one big LightCurveCollection with all of it.

In [6]:
# Load in Brian's EB list - this is a file with just one column, which is the TIC IDs of EBs in the data. They
# are not categorized by sector
brian_list = np.loadtxt('ebcat_tics.txt',dtype=int,skiprows=1)

In [7]:
brian_list

array([ 309256410,  231830630,   25154836, ..., 2025904859, 2026986390,
       2041056565])

In [8]:
# Load in sector 14 lookup file
lookup14_path = '~/tesslcs/sector14lookup.csv'
lookup14 = pd.read_csv(lookup14_path,header=None,names=['filename','RA','dec','TIC ID','sector','camera','CCD',
    'mag'],index_col=False,dtype={'filename':str,'RA':float,'dec':float,'TIC ID':int,'sector':int,'camera':int,
    'CCD':int,'mag':float},skiprows=1)

I need a good way of getting 1500-2000 TIC IDs that are present in lookup14['TIC ID'] but _not_ in brian_list.  

In [9]:
print(brian_list.size)
print(lookup14['TIC ID'].size)

470016
4009717


I can see from the above that lookup14 is about an order of magnitude larger than brian_list, so if we have to iterate through one of them with a for loop, brian_list would be preferable. What I could do is loop through brian_list, then for each element, see if it's in lookup14['TIC ID'], and if not, add it to a list of non-EBs. Another thing I could do instead is use some function to find the elements that are present in both arrays, then make a new version of lookup14 with those elements removed. 

Let me try the second one and see how it goes.

In [10]:
# Test case: I have here 2 arrays and I want to get all the elements that are in the first one but not in the 
# second one. And it seems to work! Inspired by 
# https://stackoverflow.com/questions/1388818/how-can-i-compare-two-lists-in-python-and-return-matches and
# https://stackoverflow.com/questions/5823684/python-find-numbers-not-in-set
x = np.array([2,4,6,8,10])
y = np.array([1,2,3,4,5])
set(x) - set(y)

{6, 8, 10}

In [11]:
# Now let's do it for real with our actual arrays. First I think I should p
set(lookup14['TIC ID']) - set(brian_list)

{158393592,
 159383554,
 276824072,
 436207624,
 276824074,
 436207628,
 436207629,
 385875982,
 159383567,
 276824080,
 276824084,
 276824088,
 276824093,
 436207646,
 276824094,
 385876000,
 436207649,
 276824102,
 352321577,
 436207660,
 436207661,
 436207662,
 276824112,
 385876017,
 436207667,
 352321587,
 276824119,
 436207671,
 276824121,
 385876023,
 159383608,
 276824124,
 436207678,
 276824127,
 159383620,
 276824136,
 276824139,
 436207696,
 276824148,
 352321620,
 276824150,
 436207705,
 1400897627,
 385876059,
 385876061,
 276824156,
 436207711,
 276824160,
 276824163,
 436207716,
 436207720,
 352321644,
 159383660,
 159383663,
 436207742,
 276824200,
 436207755,
 385876107,
 276824210,
 385876115,
 385876116,
 276824213,
 436207762,
 436207771,
 352321692,
 436207773,
 385876134,
 276824231,
 276824232,
 385876137,
 276824235,
 276824237,
 276824238,
 436207793,
 276824243,
 436207796,
 385876149,
 436207797,
 276824245,
 385876153,
 276824254,
 436207809,
 276824261,
 43

In [12]:
# Test: Are all the EBs that I've identified in Brian's list? They should be...
count = 0
bad_ticids = []
good_ticids = []
for ticid in ebs['TIC ID']:
    if ticid not in brian_list:
        count+=1
        bad_ticids.append(ticid)
    else:
        good_ticids.append(ticid)

In [13]:
count

972

In [14]:
bad_ticids[:5]

[272716209, 185057845, 271970252, 169819920, 390207656]

In [15]:
272716209 in brian_list

False

In [16]:
185057845 in brian_list

False

In [17]:
271970252 in brian_list

False

In [18]:
169819920 in brian_list

False

In [19]:
390207656 in brian_list

False

In [20]:
good_ticids[:5]

[267667898, 268305489, 272375892, 138037268, 184556122]

In [21]:
267667898 in brian_list

True

In [22]:
268305489 in brian_list

True

In [23]:
272375892 in brian_list

True

In [24]:
138037268 in brian_list

True

In [25]:
184556122 in brian_list

True

Why could this be? All the EBs should be in Brian's list, but I'm finding that over half of the ones in my "ebs" dataframe are not. If this is the case, then using "set(lookup14['TIC ID']) - set(brian_list)" as my non-EB training data will probably result in some EBs being labeled as non-EBs, which will throw off the model. 

How did this happen? My "ebs" dataframe came from "cleaned_sector_14_ebs_filepaths.csv". There could be a problem: 1) with the original list of Kepler EBs from keplerebs.villanova.edu; 2) with Jim Davenport's KIC2TIC.csv file (maybe some of the listed TICs are incorrect); 3) with my processing code at the end of "Kepler EBs to TIC IDs and Filepaths (Streamlined).ipynb" (but I just looked through this and it seems right to me; 4) with my code in here somehow, but this seems right to me too; or 5) with Brian's list (it's not comprehensive of all the EBs observed by TESS like I thought it was).

In [26]:
# Number of sector 14 EBs that Brian identifies
len(set(lookup14['TIC ID']) & set(brian_list))

40823

In [27]:
# Want to get the number of TIC IDs that are present in Brian's list and also present in the second column of Jim's
# KIC2TIC.csv file, because those will be the "EB"s from Brian's list that are in the Kepler FOV
# Read in Jim's file
kic2tic_path = '~/kic2tic/KIC2TIC.csv'
k2t = pd.read_csv(kic2tic_path,header=None,names=['KIC ID','TIC ID'],index_col=False,
                  dtype={'KIC ID':int,'TIC ID':int},skiprows=1)

In [28]:
# Get the number of objects in both k2t['TIC ID'] and brian_list
len(set(k2t['TIC ID']) & set(brian_list))

1755

In [29]:
k2t['TIC ID'].size

199421

I am going back and forth with Daniel right now (have been for close to an hour) to figure out this issue. He is saying I should check false positives/false negatives in Brian's data assuming that the Villanova data is perfect and that Jim's conversion (KIC to TIC) file is perfect. So he says I should get the magnitudes of Brian's data and then take only the ones that are between mags 10 and 15. How to get the mags? Use the lookup table. Let's check this out:

In [30]:
sec_14_and_brian_lookup_inds = []
sec_14_and_brian_mags = [] # TESS mag, I think, since it's from the lookup table
for ind, el in enumerate(lookup14['TIC ID']):
    if el in brian_list:
        sec_14_and_brian_mags.append(lookup14['mag'][ind])
        sec_14_and_brian_lookup_inds.append(ind)
    # Note: I don't know how many TIC IDs are listed in Brian's catalog but not in the lookup tables because I
    # would need to check all 26 lookup tables for this

KeyboardInterrupt: 

In [31]:
lookup14[lookup14['mag'] < 15]

,filename,RA,dec,TIC ID,sector,camera,CCD,mag
0,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.037464,52.358691,27693449,14,2,4,9.83679
1,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.347333,52.487350,27837522,14,2,4,9.59878
2,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.262922,52.729809,27837873,14,2,4,12.93180
3,tesslcs_sector_14_104/2_min_cadence_targets/te...,298.571902,53.778422,264393379,14,2,4,11.26480
4,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.146668,52.858682,27692689,14,2,4,10.99730
...,...,...,...,...,...,...,...,...
4009712,tesslcs_sector_14_104/tesslcs_tmag_9_10/tesslc...,266.543113,76.482123,1401093064,14,3,3,9.52726
4009713,tesslcs_sector_14_104/tesslcs_tmag_9_10/tesslc...,266.545287,76.482495,1401093065,14,3,3,9.65109
4009714,tesslcs_sector_14_104/tesslcs_tmag_9_10/tesslc...,216.211428,76.585895,156457298,14,3,3,9.82640
4009715,tesslcs_sector_14_104/tesslcs_tmag_9_10/tesslc...,285.164309,76.615343,420107253,14,3,2,9.95180


In [32]:
# The above cell shows that all the entries in lookup14 have mag < 15, so I can just put the single condition
# mag > 10 on the slice I get from this array
lookup14[lookup14['mag'] > 10]

,filename,RA,dec,TIC ID,sector,camera,CCD,mag
2,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.262922,52.729809,27837873,14,2,4,12.9318
3,tesslcs_sector_14_104/2_min_cadence_targets/te...,298.571902,53.778422,264393379,14,2,4,11.2648
4,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.146668,52.858682,27692689,14,2,4,10.9973
7,tesslcs_sector_14_104/2_min_cadence_targets/te...,238.576783,73.994667,160488880,14,3,4,12.2847
8,tesslcs_sector_14_104/2_min_cadence_targets/te...,238.514779,73.902974,160488856,14,3,4,10.9420
...,...,...,...,...,...,...,...,...
3960837,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,296.283962,16.491325,343910612,14,1,3,14.9587
3960838,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,298.593335,46.365461,274197266,14,2,4,14.7924
3960839,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,294.210346,45.482392,270955811,14,2,4,14.0506
3960840,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,300.010263,36.002688,173075879,14,1,1,14.9941


In [33]:
lookup14_magfiltered = lookup14[lookup14['mag'] > 10]

In [57]:
# Now cross-reference the stars in this table with Brian's list
brian_magfiltered_inds = []
brian_magfiltered_mags = []
for obj in enumerate(brian_list):
    index = np.where(lookup14_magfiltered['TIC ID'] == obj)[0]
    if index.size < 1:
        continue
    index = index[0]
    mag = lookup14_magfiltered['mag'][index]
    print(test)
    brian_magfiltered_inds.append(index)
    brian_magfiltered_mags.append(mag)

KeyboardInterrupt: 

In [34]:
brian_magfiltered_inds

NameError: name 'brian_magfiltered_inds' is not defined

Why is this producing zero stars? I know this can't be right. Let me make sure that there is indeed overlap between lookup14_magfiltered['TIC ID'] and brian_list:

In [34]:
len(set(lookup14_magfiltered['TIC ID']) & set(brian_list))

40036

In [35]:
len(set(lookup14['TIC ID']) & set(brian_list)) # Unfiltered by mag

40823

Yeah, there is overlap - almost the same number as with the unfiltered lookup14 table. So what's going on here? Why am I getting zero stars from my loop above?

In [36]:
lookup14_magfiltered['TIC ID']

2             27837873
3            264393379
4             27692689
7            160488880
8            160488856
              ...     
3960837      343910612
3960838      274197266
3960839      270955811
3960840      173075879
3960841    10000796624
Name: TIC ID, Length: 3952247, dtype: int64

In [37]:
(set(lookup14_magfiltered['TIC ID']) & set(brian_list))

{11796481,
 295305218,
 436469773,
 376832013,
 287178771,
 287178772,
 102498325,
 274071576,
 436469785,
 385876005,
 102498341,
 378273831,
 219807787,
 89522230,
 89522233,
 28442686,
 102498367,
 28442690,
 233177155,
 184418370,
 28442693,
 193855557,
 42205255,
 89522250,
 193855564,
 193855566,
 274071635,
 184418389,
 378404949,
 1271136343,
 87687255,
 102498391,
 193855578,
 383647836,
 237109346,
 395706469,
 12583014,
 394788968,
 89653358,
 89522288,
 159645810,
 435814516,
 12583035,
 435814523,
 89522306,
 394788994,
 230293636,
 12583050,
 426115213,
 233046159,
 284557457,
 394789011,
 462553243,
 462553244,
 233308315,
 462553247,
 233308319,
 244580515,
 233308335,
 233308336,
 290193586,
 294387891,
 198181044,
 244580533,
 12583094,
 233308340,
 290193587,
 268304568,
 1551761596,
 233308352,
 10223815,
 244580556,
 376438990,
 233308369,
 288882898,
 451805393,
 294387929,
 376439003,
 158990559,
 294387940,
 47972583,
 158990568,
 138543338,
 233570553,
 2943879

In [38]:
11796481 in brian_list

True

In [39]:
11796481 in lookup14_magfiltered['TIC ID']

False

OK, here's the issue - even though 11796481 (for example) shows up in both brian_list and lookup14_magfiltered['TIC ID'] (I know this because it is in (set(lookup14_magfiltered['TIC ID']) & set(brian_list))), the command "11796481 in lookup14_magfiltered['TIC ID']" returns False. Why is that?

In [40]:
# Try this
11796481 in lookup14_magfiltered['TIC ID'][:]

False

In [41]:
11796481 in set(lookup14_magfiltered['TIC ID'])

True

In [42]:
# OK, I don't know why this is the case. But I'm going to move on. With the set command, I can get the list of TIC
# IDs in Brian's dataset that are 1) in sector 14 and 2) between mags 10 and 15. Then I can take that list and get
# the answers to Daniel's questions: 
# 1. How many EBs are in the Villanova (mag-filtered) list and not Brian's (mag-filtered) list?
# 2. How many EBs are in Brian's (mag-filtered) list and not the Villanova (mag-filtered) list?
# 3. How many are in common?
brian_list_14_magfiltered = (set(lookup14_magfiltered['TIC ID']) & set(brian_list)) # TIC IDs
# Don't forget about the discrepancy between sector 14 and the Kepler FOV -- not the same thing!
# brian_list_14_magfiltered is a list of TIC IDs for EBs that are in Brian's list, in sector 14, and mag-filtered. 
# Now let's get a list of TIC IDs for EBs that are in Brian's list, in sector 14, in the Kepler FOV, and mag-
# filtered. 
brian_list_14_kepler_magfiltered = brian_list_14_magfiltered & set(k2t['TIC ID'])
print(len(brian_list_14_kepler_magfiltered))

1635


In [43]:
# So there are 1635 stars in Brian's list that are in the Kepler FOV, in sector 14, and between mags 10 and 15
# Now let's get the list of TIC IDs that are in the Villanova catalog, in sector 14, and between mags 10 and 15
# (Note that if a star is in the Villanova catalog, that implies it's in the Kepler FOV)
# Actually, turns out that list is already present in the cleaned_sector_14_ebs_filepaths.csv file - can access it
# with ebs['TIC ID']
print(len(ebs['TIC ID']))

1809


OK, so there are 1809 stars in Villanova's list that are in the Kepler FOV, in sector 14, and between mags 10 and 15, while there are 1635 stars in Brian's list that meet the same critera. But what is the overlap between them?

In [44]:
print(len((set(ebs['TIC ID']) & set(brian_list_14_kepler_magfiltered))))

830


Only 830 TIC IDs are shared between these 2 lists. That means that when comparing equivalent lists (in sector 14, in the Kepler FOV, and between mags 10 and 15), there are 979 stars in Villanova's list but not in Brian's list; 805 stars in Brian's list but not in Villanova's list; and 830 stars in common. Hence the answers to Daniel's 3 questions are 979, 805, 830.

**...or maybe not.** I just realized that I used the Kepler magnitude to filter the Villanova data by mag, but I used the magnitudes in the lookup tables (I'm assuming this is the TESS magnitude) to filter Brian's data by mag. So there could be TIC IDs that I've currently counted as appearing in Brian's dataset but not the Villanova dataset but they actually are in both, it's just that their Kepler mag is a little below 10 so I missed them in my dataset but their TESS mag is above 10, so they should be included. To fix this, I'm going to download the entire Villanova Kepler dataset (unfiltered by magnitude), then look up those TIC IDs in our lookup tables to get their TESS magnitudes and end up with a fully correct list of Villanova stars that are 1) in sector 14 and 2) have T-mags between 10 and 15.

#### Redoing the False Positives/Negatives with Full Unfiltered Villanova Kepler Data

In [45]:
# Load in the complete Villanova Kepler data
kepler_complete_path = '~/berkeley-seti/keplerebs_villanova_complete.csv'
keplerebs_complete = pd.read_csv(kepler_complete_path,header=None,names=['KIC ID','period','period err','bjd0','bjd0 err',
    'morphology','RA','dec','gal lon','gal lat','kmag','Teff','SC'],index_col=False,usecols=['KIC ID','period',
    'period err','RA','dec','kmag','Teff'],skiprows=8)

In [46]:
keplerebs_complete

,KIC ID,period,period err,RA,dec,kmag,Teff
0,3863594,0.053268,0.0,-1.0000,-1.0000,-1.000,-1.0
1,10417986,0.073731,0.0,296.7488,47.5438,9.128,-1.0
2,8912468,0.094838,0.0,300.1156,45.1679,11.751,6194.0
3,8758716,0.107205,0.0,293.8519,44.9494,13.531,-1.0
4,10855535,0.112782,0.0,289.2119,48.2031,13.870,7555.0
...,...,...,...,...,...,...,...
2917,9408440,989.985000,-1.0,293.5906,45.9317,13.199,5688.0
2918,8054233,1058.000000,-1.0,299.2436,43.8144,11.783,4733.0
2919,7672940,1064.270000,-1.0,288.2586,43.3765,12.328,-1.0
2920,11961695,1082.815000,-1.0,290.6239,50.3904,13.718,5768.0


In [ ]:
# Roadmap:
# Convert the KIC IDs here to TIC IDs with Jim's file
# Look up those TIC IDs in the sector 14 lookup table - we'll lose some because parts of the Kepler FOV aren't in 
# sector 14
# Get the magnitude (from the lookup table) for each of these sector 14 Villanova (Kepler) EBs
# Filter out only the ones with TESS mags between 10 and 15
# Compare the overlap in this list with brian_list_14_kepler_magfiltered

In [47]:
# Now the full Villanova Kepler data is loaded in - I need to convert all these KIC IDs to TIC IDs like I did 
# before (in the "Kepler EBs... .ipynb" file)
ticids_kepler = []
not_found_counter = 0 # Will count how many KIC IDs aren't found in the k2t dataframe
for kicid in keplerebs_complete['KIC ID']:
    inds = np.where(k2t['KIC ID'] == kicid)[0]
    if inds.size == 0:
        not_found_counter += 1
        continue
    elif inds.size > 1:
        print('More than one line present for KIC ID ' + str(kicid))
    ticids_kepler.append(k2t['TIC ID'][inds[0]]) # Add the 1st TIC ID to the list (presumably it will be the only TIC ID)
print('Number not found: ' + str(not_found_counter))

Number not found: 23


Interesting - this time there were **23 KIC IDs that were present in the Villanova data but not listed in Jim Davenport's KIC2TIC.csv file**. When I did this with only the Villanova EBs between K-mags 10 and 15, there were 0 TIC IDs not listed in Jim's file. I wonder why this is the case - casts some doubt on either the completeness of Jim's file or the reliability of Villanova's data.

In [48]:
len(ticids_kepler)

2899

In [49]:
# So far, so good. Now look up these TIC IDs in the sector 14 lookup table and get their mags
ticids_kepler = np.asarray(ticids_kepler) # Just cast the list as an array for easier processing
sector_14_eb_indices_complete = []
for lookup_index, ticid in enumerate(lookup14['TIC ID']):
    inds = np.where(ticids_kepler == ticid)[0]
    if inds.size == 0:
        continue
    elif inds.size > 1:
        print('More than one line present in ticids_kepler for TIC ID ' + str(ticid))
    sector_14_eb_indices_complete.append(lookup_index) # This time I am just appending the index so I can then make a new df

More than one line present in ticids_kepler for TIC ID 121121622
More than one line present in ticids_kepler for TIC ID 63366439
More than one line present in ticids_kepler for TIC ID 394177355
More than one line present in ticids_kepler for TIC ID 273379486
More than one line present in ticids_kepler for TIC ID 120499724
More than one line present in ticids_kepler for TIC ID 270782003
More than one line present in ticids_kepler for TIC ID 272177504
More than one line present in ticids_kepler for TIC ID 243271716
More than one line present in ticids_kepler for TIC ID 158489694
More than one line present in ticids_kepler for TIC ID 273373648
More than one line present in ticids_kepler for TIC ID 137635677


Here there are 11 TIC IDs with more than one line in ticids_kepler, as opposed to 8 with the pre-filtered by mag Villanova data. Let's check and see if this is due to duplicate listings in the Villanova data or due to more than one KIC ID pointing to the same TIC ID in Jim's file.

In [50]:
print(len(k2t['TIC ID']))
print(len(set(k2t['TIC ID'])))

199421
199421


Ah, so it is in fact due to duplicates in the Villanova data. And in fact, after scrolling through a little of the Villanova data by eye just now, I found that KIC 2856960 is repeated (it's there twice) in the Villanova list! Why would that be? Weird, but I can deal with it no problem, it's just surprising. I wonder if it's an error or if there's some real reason for it that I don't know about, because those two entries have different values listed for their other parameters.

In [51]:
print(len(ticids_kepler))
print(len(set(ticids_kepler)))

2899
2886


Interesting - filtering out duplicates gets rid of 13 entries in the Villanova catalog. But only 11 unique TIC IDs are present more than once in ticids_kepler, leading me to believe that either 2 are listed three times, or one is listed 4 times. I can test this out real quick by getting the mode of the dataset.

In [52]:
from scipy import stats
stats.mode(ticids_kepler)

ModeResult(mode=array([273373648]), count=array([3]))

So I was right - there is at least one that's in here 3 times, and I'm satisfied to believe that there's one more that's in here 3 times too (stats.mode returns only the smallest value if there's a tie for the mode).

In [53]:
# Now let's see how many of the 2886 unique TIC IDs I got from the Villanova data are present in the sector 14 
# lookup table:
print(len(sector_14_eb_indices_complete))

2238


In [54]:
# Get the (TESS) magnitudes of these 2238 EBs
sector_14_eb_indices_complete = np.asarray(sector_14_eb_indices_complete) # Make array so next line works
sector_14_eb_indices_complete_magfiltered = sector_14_eb_indices_complete[np.where(lookup14['mag'][sector_14_eb_indices_complete] > 10)[0]]

In [55]:
# Just confirming here that there are no stars with mag > 15 in the lookup table, so I don't need to include
# that condition in the above cell with sector_14_ebs_indices_complete_magfiltered 
np.where(lookup14['mag'][sector_14_eb_indices_complete] > 15)[0]

array([], dtype=int64)

In [56]:
print(len(sector_14_eb_indices_complete_magfiltered))

2172


So 2172 of the 2238 EBs are in the TESS magnitude range. It's interesting to note that this number was only 1809 when I did it for the pre-filtered Kepler data! Pretty surprising that the difference is this big - seems like over 300 EBs have Kepler mags outside of [10,15] but TESS mags within that range. Let me confirm this is true:

In [57]:
mag_discrepancy_counter = 0
for x in sector_14_eb_indices_complete_magfiltered: # Remember these are indices in lookup14
    if lookup14['mag'][x] > 10: # I know it's not over 15 because the lookup table doesn't contain any of those
        x_ticid = lookup14['TIC ID'][x]
        inds = np.where(k2t['TIC ID'] == x_ticid)[0]
        if inds.size > 0: # If this is true, then inds.size == 1 since I checked and there are no duplicates
            x_kicid = k2t['KIC ID'][inds[0]]
            kepinds = np.where(keplerebs_complete['KIC ID'] == x_kicid)[0] 
            if kepinds.size > 0: # If this KIC ID is present
                x_kmag = keplerebs_complete['kmag'][kepinds[0]]
                if ~(10 < x_kmag < 15):
                    mag_discrepancy_counter += 1
print(mag_discrepancy_counter)

384


There are indeed > 300 such EBs. There seems to be a slight discrepancy in that I would expect the number to be 2172-1809 = 363, and it's actually 384. I don't know why this is, but I'm going to say it's not super important for now. Let's move on.

Quick recap of how I got here: I first took the Villanova Kepler EBs from the website, which had 2922 elements in it. 23 of those weren't found in Jim's file, so that left me with 2899. 13 of those were duplicates, which got filtered out when I looped through the lookup table and recorded all the TIC IDs from the table that were also present in the Villanova data. But in doing that, I also got rid of all the Villanova entries that weren't in sector 14. That left me with 2238 Villanova EBs in the Kepler FOV, in sector 14. Then I took only the ones with TESS magnitudes between 10 and 15 and got 2172 left.

In [58]:
# Now get false positives/negatives with Brian's data
# Remember that the brian_list_14_kepler_magfiltered array should still be valid - just need to get the 
# corresponding array of TIC IDs from the Villanova data that are in the Kepler FOV, in sector 14, and between 
# TESS mags 10 and 15. That array is lookup14['TIC ID'][sector_14_eb_indices_complete_magfiltered]
len(set(brian_list_14_kepler_magfiltered) & set(lookup14['TIC ID'][sector_14_eb_indices_complete_magfiltered]))

942

In [59]:
print(len(set(brian_list_14_kepler_magfiltered)))
print(len(set(lookup14['TIC ID'][sector_14_eb_indices_complete_magfiltered])))

1635
2172


OK, so there are actually 2172 stars in Villanova's (complete) list that are in the Kepler FOV, in sector 14, and between TESS mags 10 and 15, while there are 1635 stars in Brian's list that meet the same criteria. The overlap between them is 942 stars. So there are 1230 stars in the Villanova list but not in Brian's list; 693 stars in Brian's list but not in the Villanova list; and 942 stars in common. Hence the answers to Daniel's 3 questions (see Slack communication with him from the afternoon of 7/12) are 1230, 693, and 942.

#### Back to the Main Work...
OK, that was a shockingly long detour to learn a little about Brian's dataset because I was really surprised when I found that more than half of the EBs I had weren't in there. What I found is there's actually a non-negligible amount of extra EBs present when I filter the Villanova catalog by TESS magnitude (lookup tables) rather than Kepler magnitude. So I'm going to roll with the keplerebs_complete list that I introduce in this notebook rather than the pre-filtered one that I used in "Kepler EBs to TIC IDs and Filepaths.ipynb".

It seems like scikit-learn's train_test_split function can take care of splitting the data into training and test for me, so I don't need to do that myself. What I do need to do is get my non-EB training data. I was initially going to just take a random sample of the TIC IDs that appeared in the sector 14 lookup table but not in Brian's catalog, but I now know that his catalog isn't reliable enough for me to do that (there's a lot of EBs that don't appear in it). So instead I'll be conservative and take a random sample of the TIC IDs that appear in the sector 14 lookup table but not in _either_ Brian's catalog or the Villanova catalog. 

In [109]:
# So what I want to do is take the lookup table (lookup14['TIC ID']) and "subtract out" (remove) the TIC IDs that 
# appear in either brian_list or ticids_kepler
nonebs = set(lookup14['TIC ID']) - set(brian_list) - set(ticids_kepler)
print(len(nonebs))

3967619


In [110]:
# Plan is to basically have 2 dataframes - one for my non-EBs and one for my EBs, then merge them
nonebs_sample = np.random.choice(np.asarray(list(nonebs)),size=2000,replace=False) # These are TIC IDs
nonebs_lookup_inds = []
for el in nonebs_sample:
    nonebs_lookup_inds.append(np.where(lookup14['TIC ID'] == el)[0][0])
nonebs_df = lookup14.iloc[nonebs_lookup_inds]
nonebs_df = nonebs_df.assign(label=np.full(nonebs_sample.size,'Non-EB')) # Add a column with the label ('Non-EB')
nonebs_df

,filename,RA,dec,TIC ID,sector,camera,CCD,mag,label
3601132,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,292.477134,39.606102,137631270,14,1,2,14.7003,Non-EB
2432723,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,286.437528,27.041996,281774671,14,1,2,14.9556,Non-EB
157137,tesslcs_sector_14_104/tesslcs_tmag_11_12/tessl...,176.429848,80.224752,365929429,14,4,2,11.6336,Non-EB
333777,tesslcs_sector_14_104/tesslcs_tmag_12_13/tessl...,296.055775,26.025087,1855967387,14,1,3,12.9747,Non-EB
3827209,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,294.084230,26.322092,213493318,14,1,3,14.6636,Non-EB
...,...,...,...,...,...,...,...,...,...
3886867,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,252.935993,78.684464,235711598,14,3,3,14.6850,Non-EB
2374513,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,281.882611,59.606423,232647249,14,2,1,14.7719,Non-EB
2214836,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,301.037205,28.679997,244291078,14,1,4,14.6852,Non-EB
2494762,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,301.169427,24.212364,244361786,14,1,4,14.0170,Non-EB


Note that the non-EB data is not filtered by magnitude, but I think that's OK.

In [111]:
ebs_df = lookup14.iloc[sector_14_eb_indices_complete_magfiltered]
ebs_df = ebs_df.assign(label=np.full(ebs_df['TIC ID'].size,'EB'))
ebs_df

,filename,RA,dec,TIC ID,sector,camera,CCD,mag,label
283,tesslcs_sector_14_104/2_min_cadence_targets/te...,291.048816,36.839902,122784501,14,1,2,11.5170,EB
1304,tesslcs_sector_14_104/2_min_cadence_targets/te...,283.259130,42.845596,164463075,14,2,3,11.5197,EB
1708,tesslcs_sector_14_104/2_min_cadence_targets/te...,298.092640,48.104488,28359831,14,2,4,10.8330,EB
3413,tesslcs_sector_14_104/2_min_cadence_targets/te...,298.371277,47.813849,273874851,14,2,4,11.4040,EB
3415,tesslcs_sector_14_104/2_min_cadence_targets/te...,296.572435,47.752712,272598447,14,2,4,11.2580,EB
...,...,...,...,...,...,...,...,...,...
3948606,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,286.764965,43.796328,158318596,14,2,3,14.5651,EB
3949021,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,290.996435,43.458738,159577379,14,2,3,14.3283,EB
3949870,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,281.649971,42.132043,123364486,14,2,3,14.6427,EB
3950085,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,283.201188,43.888972,164459948,14,2,3,14.0278,EB


In [112]:
# Combine these 2 dataframes
final_input_df = ebs_df.append(nonebs_df)
final_input_df = final_input_df.sample(frac=1) # Shuffles the dataframe
final_input_df

,filename,RA,dec,TIC ID,sector,camera,CCD,mag,label
510886,tesslcs_sector_14_104/tesslcs_tmag_12_13/tessl...,299.426873,46.700156,268934915,14,2,4,12.8903,EB
3561462,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,301.506542,31.797206,90003106,14,1,1,14.9186,Non-EB
1785330,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,296.658850,42.369743,272709670,14,2,4,13.6568,EB
1035490,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,307.230318,28.332637,436364347,14,1,4,13.1475,Non-EB
273325,tesslcs_sector_14_104/tesslcs_tmag_11_12/tessl...,291.003492,38.526853,122684183,14,1,2,11.8381,EB
...,...,...,...,...,...,...,...,...,...
2628147,tesslcs_sector_14_104/tesslcs_tmag_14_15/tessl...,299.301885,20.839294,263369443,14,1,3,14.4938,Non-EB
1557646,tesslcs_sector_14_104/tesslcs_tmag_13_14/tessl...,296.887867,26.042990,452218321,14,1,3,13.1081,Non-EB
718893,tesslcs_sector_14_104/tesslcs_tmag_12_13/tessl...,288.327181,46.471706,158730053,14,2,3,12.2458,EB
273326,tesslcs_sector_14_104/tesslcs_tmag_11_12/tessl...,291.018796,38.569840,122684284,14,1,2,11.6844,EB


Now I have my input data, and it has the training labels on it. So I think the next thing to do is read in the lightcurves corresponding to all my filenames into a LightCurveCollection object.

In [113]:
# Add "/home/parsellsx/tesslcs" to the beginning of every one of the filenames in the dataframe
# Note: You CANNOT do "~/tesslcs" - if you do, it doesn't recognize the files
lccoll = LightCurveCollection(list(final_input_df['sector']),list(final_input_df['camera']),['/home/parsellsx/tesslcs/' + s for s in list(final_input_df['filename'])],factor=10)

In [114]:
lccoll.load_all_lcs() # Necessary step for Daniel's loading mechanism (loaders.py)
lcs = lccoll.lcs

In [115]:
# I'm not really sure what form the lcs lightcurves are in - let me see:
lcs

[<TessLightCurve length=1019 LABEL="TIC 268934915">
        time               flux             flux_err      quality
                                                                 
       object            float64            float64        int64 
 ------------------ ------------------ ------------------ -------
 1683.3880789140453 1.0256292206590913 0.9933572466876608       0
  1683.513081206327 1.0150362401770703 0.9929238688172644       0
 1683.6589171241374 1.0147332140726597 0.9908682094421658       0
 1683.7005845148535 1.0065506179510006 0.9913549309232191       0
 1683.7839192869367 1.0050299558749234  0.991452673388028       0
 1684.0339236517354  0.946294506285314  0.981416464614124       0
 1684.0755910645403  1.001045743485436  0.989090072102889       0
 1684.2214270453346 1.0091330806551841 0.9951165885287083       0
 1684.2422607591463  1.013233805659699 0.9999473677135845       0
                ...                ...                ...     ...
 1709.9716593970231 0.98

OK interesting, so it keeps them in the same order as they were in in the final_input_df dataframe. That means I should be able to easily feed in final_input_df['label'] as the training labels, which is good. Let me check and see how to access a single lightcurve:

In [116]:
lcs[0]

time,flux,flux_err,quality
,,,
object,float64,float64,int64
1683.3880789140453,1.0256292206590913,0.9933572466876608,0
1683.513081206327,1.0150362401770703,0.9929238688172644,0
1683.6589171241374,1.0147332140726597,0.9908682094421658,0
1683.7005845148535,1.0065506179510006,0.9913549309232191,0
1683.7839192869367,1.0050299558749234,0.991452673388028,0
1684.0339236517354,0.946294506285314,0.981416464614124,0
1684.0755910645403,1.001045743485436,0.989090072102889,0
1684.2214270453346,1.0091330806551841,0.9951165885287083,0


In [117]:
lcs[0]['flux']

1.0256292206590913
1.0150362401770703
1.0147332140726597
1.0065506179510006
1.0050299558749234
0.946294506285314
1.001045743485436
1.0091330806551841
1.013233805659699
1.010295732651024
1.015499725737221


In [118]:
lcs[0]['flux'][0]

1.0256292206590913

In [119]:
lcs[0]['flux'][:5]

1.0256292206590913
1.0150362401770703
1.0147332140726597
1.0065506179510006
1.0050299558749234


In [78]:
lcs['flux']

TypeError: list indices must be integers or slices, not str

The slicing/indexing works mostly like I'd expect it to. Now I think it's time for me to featurize all the lightcurves.

In [120]:
# First I want to check and make sure that quality column is equal to 0 everywhere, because it should be
for x in lcs:
    if np.count_nonzero(x['quality']) > 0:
        print('Err')

No "Err"s, so we're good to go on the quality front.

In [121]:
# Test out a list comprehension for passing in to the "times" parameter of the featurizer (next cell)
print([lc['time'] for lc in lcs][:5]) # This has some other metadata of the Time object
print([lc['time'].value for lc in lcs][:5]) # This is what I want - just the actual time values

[<Time object: scale='tdb' format='btjd' value=[1683.38807891 1683.51308121 1683.65891712 ... 1710.1174928  1710.13832614
 1710.15915949]>, <Time object: scale='tdb' format='btjd' value=[1683.36854813 1683.38938187 1683.41021562 1683.43104936 1683.47271684
 1683.49355057 1683.53521803 1683.66022035 1683.68105406 1683.70188778
 1683.74355519 1683.78522261 1683.80605632 1683.82689002 1683.84772373
 1683.86855744 1683.93105857 1683.95189229 1683.972726   1683.99355972
 1684.01439344 1684.03522716 1684.0768946  1684.11856205 1684.13939578
 1684.16022951 1684.18106324 1684.20189697 1684.2227307  1684.26439817
 1684.2852319  1684.30606563 1684.32689936 1684.34773309 1684.36856681
 1684.38940054 1684.41023426 1684.43106798 1684.4519017  1684.47273542
 1684.51440284 1684.53523655 1684.55607025 1684.57690396 1684.59773766
 1684.61857136 1684.63940505 1684.66023874 1684.68107244 1684.70190613
 1684.72273981 1684.7435735  1684.78524088 1684.80607456 1684.82690825
 1684.84774194 1684.86857562 1684

[array([1683.38807891, 1683.51308121, 1683.65891712, ..., 1710.1174928 ,
       1710.13832614, 1710.15915949]), array([1683.36854813, 1683.38938187, 1683.41021562, 1683.43104936,
       1683.47271684, 1683.49355057, 1683.53521803, 1683.66022035,
       1683.68105406, 1683.70188778, 1683.74355519, 1683.78522261,
       1683.80605632, 1683.82689002, 1683.84772373, 1683.86855744,
       1683.93105857, 1683.95189229, 1683.972726  , 1683.99355972,
       1684.01439344, 1684.03522716, 1684.0768946 , 1684.11856205,
       1684.13939578, 1684.16022951, 1684.18106324, 1684.20189697,
       1684.2227307 , 1684.26439817, 1684.2852319 , 1684.30606563,
       1684.32689936, 1684.34773309, 1684.36856681, 1684.38940054,
       1684.41023426, 1684.43106798, 1684.4519017 , 1684.47273542,
       1684.51440284, 1684.53523655, 1684.55607025, 1684.57690396,
       1684.59773766, 1684.61857136, 1684.63940505, 1684.66023874,
       1684.68107244, 1684.70190613, 1684.72273981, 1684.7435735 ,
       1684.78524

In [122]:
# I wonder if the flux (which I'll pass into the "values" parameter) is also an object or just the values
print([lc['flux'] for lc in lcs][:5])
print([lc['flux'].value for lc in lcs][:5])
print([lc['flux_err'].value for lc in lcs][:5])

[<QColumn name='flux' dtype='float64' length=1019>
1.0256292206590913
1.0150362401770703
1.0147332140726597
1.0065506179510006
1.0050299558749234
 0.946294506285314
 1.001045743485436
1.0091330806551841
 1.013233805659699
 1.010295732651024
 1.015499725737221
1.0149234118962087
               ...
0.9946683169538049
0.9868443765454299
0.9811591561725733
0.9791386309800834
0.9789425038681414
0.9840465175350519
0.9831168797479478
  0.96668014905503
0.9532489810201981
0.9539870044980482
0.9689449539009801
0.9842179664885168, <QColumn name='flux' dtype='float64' length=986>
1.0200888525992706
 1.008856477911585
0.9972256257924983
 1.001686581686761
1.0129718277414472
1.0187664791154927
1.0163432251545685
1.0202281697308273
1.0084046726421665
 1.012149360152441
 1.013144295643073
 1.037662788265386
               ...
0.9837834912773434
1.0132746920154443
1.0152417402823424
0.9803634169111327
0.9990718132183488
1.0053671508115227
0.9787952778788103
0.9810463059754251
0.9692625018724298
0.9618

In [123]:
# Cool, so now I know what to feed into the featurizer
times = [lc['time'].value for lc in lcs]
values = [lc['flux'].value for lc in lcs]
errors = [lc['flux_err'].value for lc in lcs]

In [124]:
# I mostly chose these by reading the descriptions and choosing ones that sounded useful for EBs. But I'm sure
# I could do a lot better if I knew more about them, or by just experimenting
features_to_use = ['amplitude','flux_percentile_ratio_mid20','flux_percentile_ratio_mid35',
                   'flux_percentile_ratio_mid50','flux_percentile_ratio_mid65','flux_percentile_ratio_mid80', 
                   'percent_beyond_1_std','period_fast','skew','std','weighted_average',
                   'fold2P_slope_10percentile','fold2P_slope_90percentile','freq1_freq','freq2_freq','freq3_freq',
                   'freq1_amplitude1','freq1_amplitude2','freq2_amplitude1','freq2_amplitude2','freq3_amplitude1',
                   'freq3_amplitude2','freq_model_max_delta_mags','freq_signif_ratio_21','freq_signif_ratio_31',
                   'freq_varrat','freq_y_offset','linear_trend','p2p_ssqr_diff_over_var']
# Featurize
fset = featurize.featurize_time_series(times=times,values=values,errors=errors,features_to_use=features_to_use)

In [125]:
fset

feature,amplitude,flux_percentile_ratio_mid20,flux_percentile_ratio_mid35,flux_percentile_ratio_mid50,flux_percentile_ratio_mid65,flux_percentile_ratio_mid80,percent_beyond_1_std,period_fast,skew,std,...,freq2_amplitude2,freq3_amplitude1,freq3_amplitude2,freq_model_max_delta_mags,freq_signif_ratio_21,freq_signif_ratio_31,freq_varrat,freq_y_offset,linear_trend,p2p_ssqr_diff_over_var
channel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.147477,0.027686,0.061134,0.139866,0.380408,0.762182,0.157017,0.943707,-1.800833,0.074742,...,0.000315,0.006720,0.000626,2.227777e-01,0.557247,0.492367,0.000189,0.000863,-0.000096,0.133006
1,0.271232,0.125270,0.229506,0.360154,0.505942,0.741434,0.304260,7.030059,-1.219033,0.033004,...,0.001220,0.004076,0.000401,2.738906e-10,0.702989,0.471658,0.000332,0.000504,-0.000008,0.724451
2,0.019973,0.155429,0.270179,0.396614,0.549602,0.761682,0.315015,7.391204,-0.090439,0.006253,...,0.000142,0.001819,0.000557,3.092731e-03,0.524408,0.578947,0.000018,0.000152,0.000009,0.305386
3,0.647317,0.096669,0.167243,0.266520,0.378052,0.601497,0.143147,3.193524,-6.803797,0.072301,...,0.001266,0.004495,0.000310,3.092526e-11,0.878997,0.637711,0.000599,0.001078,-0.000168,0.507704
4,0.097432,0.172465,0.307842,0.448582,0.600992,0.779893,0.327586,0.734223,-4.038254,0.018738,...,0.000344,0.002454,0.000244,1.485219e-02,0.472128,0.332959,0.000066,-0.000162,-0.000037,0.394442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4167,0.370781,0.113816,0.214648,0.341702,0.491092,0.704121,0.044625,8.080899,-7.670332,0.067641,...,0.000124,0.001327,0.000104,3.042112e-11,0.809644,0.720744,0.000456,-0.000074,-0.000046,0.311539
4168,0.983793,0.047981,0.085120,0.135523,0.208295,0.328064,0.064748,2.854812,3.742587,0.092275,...,0.000220,0.003088,0.000159,2.360124e-11,0.889411,0.862026,0.002074,0.000040,-0.000117,1.019358
4169,0.008264,0.176532,0.312285,0.450796,0.616435,0.820316,0.339549,0.251610,-0.112105,0.002560,...,0.000269,0.000452,0.000227,2.982201e-11,1.070320,0.604045,0.000003,0.000008,0.000027,0.361981


In [142]:
# Save fset to CSV file. Also save final_input_df, that way I can recover which ones are EBs and which ones aren't
fset.to_csv('sector_14_ebs_nonebs_features.csv',index=False)
final_input_df.to_csv('sector_14_ebs_nonebs_final_input_df.csv',index=False)

In [127]:
# What does the fset dataframe behave like? Should be just a normal df but let me make sure
fset['amplitude']

channel,0
0,0.147477
1,0.271232
2,0.019973
3,0.647317
4,0.097432
...,...
4167,0.370781
4168,0.983793
4169,0.008264
4170,0.112794


In [128]:
fset['amplitude'][0]

0       0.147477
1       0.271232
2       0.019973
3       0.647317
4       0.097432
          ...   
4167    0.370781
4168    0.983793
4169    0.008264
4170    0.112794
4171    0.040114
Name: 0, Length: 4172, dtype: float64

In [129]:
fset['amplitude'][0][0]

0.14747662448568666

Interesting, so I have to do an additional [0] to refer to channel 0, even though there's only one channel in this data. Good to know. I also want to check and see if it's already scaled from 0 to 1 or not.

In [132]:
print(max(fset['amplitude'][0]))
print(min(fset['amplitude'][0]))

119.70378458641441
0.0004917972266885728


OK, it's not, so I will need to do a scikit-learn StandardScaler on it. Edit: Turns out that's actually not what a StandardScaler does, that's the job of a minmaxscaler. But I'll use StandardScaler here anyway.

Now the data is featurized - what next? Now I want to actually train a model, at long last! 

In [136]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

According to this link (https://scikit-learn.org/stable/getting_started.html#pipelines-chaining-pre-processors-and-estimators), it looks like you can do the train_test_split, then make a pipeline with the StandardScaler and the LinearSVC all in one that you feed the training and test data into. So let me do that.

In [137]:
train, test = train_test_split(np.arange(len(final_input_df['label'])),train_size=0.5)

In [164]:
# Get X and y for both training and testing data
X_train = fset.iloc[train]
y_train = final_input_df['label'].iloc[train]
X_test = fset.iloc[test]
y_test = final_input_df['label'].iloc[test]

In [165]:
# Make a pipeline that includes both a StandardScaler and the classifier itself
svc = make_pipeline(StandardScaler(),svm.LinearSVC(dual=False))

In [166]:
# Train the model - finally!
svc.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(dual=False))])

In [167]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [168]:
# Test
y_pred = svc.predict(X_test)

In [171]:
# Get accuracy score and precision score
print(accuracy_score(y_test,y_pred)) # Fraction of classifications that were correct
print(precision_score(y_test,y_pred,pos_label='EB')) # Fraction of positive (EB) classifications that were correct
# To be more specific, precision_score returns the ratio tp/(tp+fp), where tp = true positive, fp = false pos.
# and a positive is defined as an "EB" classification

0.7636625119846596
0.8035892323030908


Finally, finally, finally got my first ML results! 